In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from RotMNIST import RotMNIST 
from torchvision.transforms import ToTensor, Lambda
import torch.nn.functional as F

### Instatiate RotMNIST and verify behaviour below with the dataloaders
dataset_rot = RotMNIST(
    root = 'data',
    download=True,
    train=True,
    transform=torchvision.transforms.Compose(
        [torchvision.transforms.Resize(32), torchvision.transforms.ToTensor()]
    ),
    rotation_mirroring=True
)

test_dataset_rot = RotMNIST(
    root = 'data',
    download=True,
    train=False,
    transform=torchvision.transforms.Compose(
        [torchvision.transforms.Resize(32), torchvision.transforms.ToTensor()]
    ),
    rotation_mirroring=True
)

dataset_upright = RotMNIST(
    root = 'data',
    download=True,
    train=True,
    transform=torchvision.transforms.Compose(
        [torchvision.transforms.Resize(32), torchvision.transforms.ToTensor()]
    ),
    rotation_mirroring=False,
)

### Instantiate dataloader for RotMNIST and get batches
train_dataloader_rot = DataLoader(dataset_rot, batch_size=64, shuffle=True)
test_dataloader_rot = DataLoader(test_dataset_rot, batch_size=64, shuffle=True)
train_dataloader_upright = DataLoader(dataset_upright, batch_size=64, shuffle=True)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(2, 2)
        # self.linear_relu_stack = nn.Sequential(
        #     nn.Linear(32 * 5 * 5, 10),
        #     nn.ReLU(),
        
        self.fc1 = nn.Linear(32 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)

model = NeuralNetwork()

print(model)
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of trainable params: " + str(pytorch_total_params))

NeuralNetwork(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1152, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)
Number of trainable params: 42026
C:\Users\Jack\anaconda3\envs\reu-code\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [2]:
learning_rate = 0.001
batch_size = 64
epochs = 25

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss for backprop
        pred = model(X.to(device))
        loss = loss_fn(pred, y.to(device))

        # Backpropagation by setting grad to zero, calculating using backprop engine and stepping (using learning rate)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0
    
    # No gradient on training data (faster computation and no optimization happening here anyway)
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.to(device))
            test_loss += loss_fn(pred, y.to(device)).item()
            correct += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader_rot, model, loss_fn, optimizer)
    test_loop(test_dataloader_rot, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.319508  [    0/60000]
loss: 2.300205  [ 6400/60000]
loss: 2.320050  [12800/60000]
loss: 2.318786  [19200/60000]
loss: 2.313833  [25600/60000]
loss: 2.318187  [32000/60000]
loss: 2.304376  [38400/60000]
loss: 2.307965  [44800/60000]
loss: 2.305310  [51200/60000]
loss: 2.285847  [57600/60000]
Test Error: 
 Accuracy: 6.5%, Avg loss: 0.036208 

Epoch 2
-------------------------------
loss: 2.301045  [    0/60000]
loss: 2.296940  [ 6400/60000]
